In [1]:
#Importing Libraries
import random
import string # to process standard python strings
import warnings
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only

[nltk_data] Downloading package punkt to C:\Users\Shivang
[nltk_data]     Shrivastav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Shivang
[nltk_data]     Shrivastav\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import json
import csv

In [4]:
# Opening JSON file
f = open('Sample Corpus - GL Bot.json',)
df = json.loads(f.read())
bot_df = df['intents']

In [5]:
# open a file for writing
GL_Bot = open('GLBot.txt', 'w')

In [6]:
# create the csv writer object
csvwriter = csv.writer(GL_Bot)
count = 0
for value in bot_df:
     if count == 0:
            header = value.keys()
            csvwriter.writerow(header)
            count += 1
csvwriter.writerow(value.values())
GL_Bot.close()

In [7]:
dff=open('GLBot.txt','r',errors = 'ignore')
raw=dff.read()
raw = raw.lower()# converts to lowercase

In [8]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences
word_tokens = nltk.word_tokenize(raw)# converts to list of words
sent_tokens[:2]

['tag,patterns,responses,context_set\n\nticket,"[\'my problem is not solved\', \'you did not help me\', \'not a good solution\', \'bad solution\', \'not good solution\', \'no help\', \'wasted my time\', \'useless bot\', \'create a ticket\']",[\'tarnsferring the request to your pm\'],']

In [9]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [10]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [11]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [12]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit,type Bye!")
while(flag==True):
    user_response = input()
    print(user_response)
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit,type Bye!


 hi


hi
ROBO: hi there


 i am fine


i am fine
ROBO: I am sorry! I don't understand you


 hell


hell
ROBO: I am sorry! I don't understand you


 thanks


thanks
ROBO: You are welcome..
